In [1]:
import os
import time
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import  NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementClickInterceptedException

In [2]:
service = Service(executable_path="/Users/jeanettepoh/Desktop/chromedriver")
driver = webdriver.Chrome()

In [3]:
page_url = "https://sprs.parl.gov.sg/search/home"
driver.get(page_url)

In [4]:
# Get search box and fill it up
wait = WebDriverWait(driver, 10)  # Wait up to 10 seconds
search = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#divmpscreen2 > div.row > div:nth-child(1) > div > div:nth-child(1) > input')))
search.send_keys('COS')

In [5]:
# This will select the 13th parliament
wait = WebDriverWait(driver, 10)  # Wait up to 10 seconds
session = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#divmpscreen2 > div.row > div:nth-child(1) > div > div.form-group.byParText > select > option:nth-child(14)')))
session.click()

In [6]:
# Find submit element and click
wait = WebDriverWait(driver, 20)

# Find the submit button element
submit = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#divmpscreen2 > div.row > div.col-sm-12.text-right.pull-right > div > button:nth-child(2)')))

# Click using JavaScript
driver.execute_script("arguments[0].click();", submit)

In [7]:
# Create empty dictionary to store results
res_dict = {}

# Switch window and check for number of search results
driver.switch_to.window(driver.window_handles[1])
wait = WebDriverWait(driver, 60)
num_results = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#searchResults > div:nth-child(1) > div')))
res = num_results.text.split(' ')
num_clicks = int(res[-1]) // int(res[-3]) + 1

print('There are {} pages to click through'.format(num_clicks))

There are 10 pages to click through


In [8]:
# Define the directory to save the files
home_dir = os.getcwd() 
save_dir = os.path.join(home_dir, 'scraped_data')

# Create the directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

In [9]:
# Nested for loop to click through all search results
for click in range(num_clicks):

    # Assuming you start from the first window
    driver.switch_to.window(driver.window_handles[0])

    # Iterate through each search result item
    for item in range(1, 21):  # Assuming there are up to 20 items per page
        try:
            # Switch to search results page if not already there
            driver.switch_to.window(driver.window_handles[1])

            # Construct XPath for the current item
            xpath = '//*[@id="searchResults"]/table/tbody[{}]/tr[{}]/td[2]/a'.format(item, 1)
            
            # Wait for the element to be clickable and then click it
            elem = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, xpath)))
            driver.execute_script("arguments[0].scrollIntoView();", elem)
            driver.execute_script("arguments[0].click();", elem)

        except:
            break

        # Switch to the newly opened page
        driver.switch_to.window(driver.window_handles[2])

        # Print and process the current URL
        current_url = driver.current_url.split('/')[-1]
        item_key = current_url.replace('?', '_')

        # Append result to dictionary for later processing
        res_dict[item_key] = driver.page_source

        file_path = os.path.join(save_dir, f'{item_key}.txt')
        with open(file_path, encoding='utf-8', mode='w+') as file:
            file.write(driver.page_source)
        
        # Close tab
        driver.close()

    # Switch back to search results tab
    driver.switch_to.window(driver.window_handles[1])

    # Click on next page once 20 results have been saved
    try:
        next_page_xpath = '//*[@id="searchResults"]/div[3]/section/ul/li[3]/a/em'
        next_page = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.XPATH, next_page_xpath)))
    except Exception as e:
        next_page_xpath = '//*[@id="searchResults"]/div[3]/section/ul/li[1]/a/em'
        next_page = WebDriverWait(driver, 60).until(EC.element_to_be_clickable((By.XPATH, next_page_xpath)))
    
    # Scroll to the element to make sure it's visible
    driver.execute_script("arguments[0].scrollIntoView();", next_page)

    # Click the next page button
    driver.execute_script("arguments[0].click();", next_page)

    # Wait for the new page to load (adjust sleep time as necessary)
    time.sleep(5)

# Quit the driver after processing
driver.quit()